In [3]:
import elasticsearch

host = 'localhost'
es = elasticsearch.Elasticsearch(host=host, port=9200)


## Index Operations:

In [11]:
indices = elasticsearch.client.IndicesClient(es)

#Create an index
index_name = 'school'
try:
    res = indices.create(index=index_name)
except Exception as e:
    print(e)
    
#Delete an index
index_name = 'school'
#try...except... is omitted
#res = indices.delete(index=index_name)

doc_type = 'teacher' #Type of doc

## Basic CRUD operations:

In [16]:
#Get all documents
from pprint import pprint
def get_all():
    body = {
        'query': {
            'match_all': {},
        },
        'size': 1000,
    }
    
    res = es.search(index=index_name, doc_type=doc_type, body=body)
    pprint(res.get('hits').get('hits'))

def delete_all():
    body = {
        'query': {
            'match_all': {},
        },
    }
    res = es.delete_by_query(index=index_name, doc_type=doc_type, body=body)

In [28]:
delete_all()

In [39]:
#Create a document
obj_body = {
    'name': '吴芳',
    'age': 28,
    'introduction': '这位是吴芳教师，职称：助教，年龄：28',
    't_id': '5530eac6a8075110c87a67bf6caa8142',
    'title': '助教',
}

res = es.create(index=index_name, doc_type=doc_type, id=2, body=obj_body)

#Create from JSON file
file_name = 'data/data.json' #A list of dicts
with open(file_name, 'r', encoding='utf8') as f:
    import json
    data = json.load(f)
    
for ind, val in enumerate(data):
    try:
        pass
        #es.create(index=index_name, doc_type=doc_type, id=ind, body=val)
    except Exception as e:
        print(e)
        print('Create No. {} doc ERROR!'.format(ind))


In [30]:
get_all()

[{'_id': '0',
  '_index': 'school',
  '_score': 1.0,
  '_source': {'age': 43,
              'introduction': '这位是赵重阳教师，职称：讲师，年龄：43',
              'name': '赵重阳',
              't_id': 'f8ff7ac54cdf738ce70c0d3aa1dc2392',
              'title': '讲师'},
  '_type': 'teacher'},
 {'_id': '5',
  '_index': 'school',
  '_score': 1.0,
  '_source': {'age': 28,
              'introduction': '这位是吴芳教师，职称：助教，年龄：28',
              'name': '吴芳',
              't_id': '5530eac6a8075110c87a67bf6caa8142',
              'title': '助教'},
  '_type': 'teacher'},
 {'_id': '8',
  '_index': 'school',
  '_score': 1.0,
  '_source': {'age': 30,
              'introduction': '这位是吴芳教师，职称：讲师，年龄：30',
              'name': '吴芳',
              't_id': '5c4b2c1523331e1a1a997788d146804f',
              'title': '讲师'},
  '_type': 'teacher'},
 {'_id': '9',
  '_index': 'school',
  '_score': 1.0,
  '_source': {'age': 36,
              'introduction': '这位是赵刚教师，职称：讲师，年龄：36',
              'name': '赵刚',
              't_id': 'c3ca66

In [32]:
#Get a document
#Get by id
doc_id = 3
res = es.get(index=index_name, doc_type=doc_type, id=doc_id)
pprint(res)

{'_id': '3',
 '_index': 'school',
 '_source': {'age': 33,
             'introduction': '这位是钱建国教师，职称：讲师，年龄：33',
             'name': '钱建国',
             't_id': '4a68ebf86044425ccb184401165b809e',
             'title': '讲师'},
 '_type': 'teacher',
 '_version': 5,
 'found': True}


In [40]:
#Delete a document
#Delete by id
doc_id = 2
es.delete(index=index_name, doc_type=doc_type, id=doc_id)

try:
    res = es.get(index=index_name, doc_type=doc_type, id=doc_id)
except Exception as e:
    print(e)
    
#Delete by query
#query will be specified in the following chapters
query_body = {
    'query': {
        'match': {
            'name': '孙明',
        }
    }
}
es.delete_by_query(index=index_name, doc_type=doc_type, body=query_body)

try:
    res = es.get(index=index_name, doc_type=doc_type, id=7)
except Exception as e:
    print(e)

GET http://10.3.242.236:9200/school/teacher/2 [status:404 request:0.003s]
GET http://10.3.242.236:9200/school/teacher/7 [status:404 request:0.002s]


TransportError(404, '{"_index":"school","_type":"teacher","_id":"2","found":false}')
TransportError(404, '{"_index":"school","_type":"teacher","_id":"7","found":false}')


In [41]:
#Update a document
#Update by id
doc_id = 1
update_body = {
    'doc': {
        'age': 53,
    },
}
res = es.get(index=index_name, doc_type=doc_type, id=doc_id)
print('before update:')
pprint(res)
es.update(index=index_name, doc_type=doc_type, id=doc_id, body=update_body)
res = es.get(index=index_name, doc_type=doc_type, id=doc_id)
print('after update:')
pprint(res)

before update
{'_id': '1',
 '_index': 'school',
 '_source': {'age': 33,
             'introduction': '这位是王芳教师，职称：讲师，年龄：33',
             'name': '王芳',
             't_id': '92ebfe7b627a193e3f4b83c8a8ee3491',
             'title': '讲师'},
 '_type': 'teacher',
 '_version': 5,
 'found': True}
after update
{'_id': '1',
 '_index': 'school',
 '_source': {'age': 53,
             'introduction': '这位是王芳教师，职称：讲师，年龄：33',
             'name': '王芳',
             't_id': '92ebfe7b627a193e3f4b83c8a8ee3491',
             'title': '讲师'},
 '_type': 'teacher',
 '_version': 6,
 'found': True}


In [49]:
#Update by query (Omitted)


Data generator can be refered [here](#ref1)

<a id='ref1'></a>
## Reference 1: Generate data

In [ ]:
import json
import random
import hashlib
from pprint import pprint

prof_title_list = ['助教', '讲师', '副教授', '教授']

family_name_list = ['赵', '钱', '孙', '李', '周', '吴', '郑', '王']
first_name_list = ['明', '建国', '重阳', '芳', '刚', '玉']

age_list = list(range(24, 44))

NUM = 10

teacher_list = []
md5 = hashlib.md5()

for i in range(NUM):
    t_name = random.choice(family_name_list) + random.choice(first_name_list)
    t_title = random.choice(prof_title_list)
    t_age = random.choice(age_list)
    t_intro = "这位是{}教师，职称：{}，年龄：{}".format(t_name, t_title, t_age)
    md5.update(''.join([t_name, t_title, str(t_age)]).encode())
    t_id = md5.hexdigest()
    teacher = {
        't_id': t_id,
        'title': t_title,
        'name': t_name,
        'age': t_age,
        'introduction': t_intro,
    }
    teacher_list.append(teacher)

file_name = 'data.json'
with open(file_name, 'w', encoding='utf8') as f:
    json.dump(teacher_list, f, ensure_ascii=False, indent=2)
